<div style="background-color: #e7f3fe; border-left: 6px solid #2196F3; padding: 10px; margin: 10px 0; color: #000;">
    <strong>A project:</strong> This is a study of new methods for me. Created by <strong> Julliane Reyes </strong> for the completion of requirements in DS150.
</div>

 # <strong> Phase 1: Library Installs and Imports <strong>
 ***

In [1]:
%pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [1]:
import yfinance as yf
#---------------------------------------------------
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
''' 
A .py file containing the machine learning models for the project. 
Just an experimentation to be honest.
'''

from Models.RY_MachineLearningModels import (
    LinearReg_model,
    AuReg_model,
    MovAve_model,
    ARIMA_model,
    SARIMA_model,
    GARCH_model,
    LSTM_model
)

from Models.RY_MLEvaluation import (
    LinearReg_eval,
    AuReg_eval,
    MovAve_eval,
    ARIMA_eval,
    SARIMA_eval,
    GARCH_eval,
    LSTM_eval
)

In [5]:
# Required Libraries ---------------------------
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from arch import arch_model #GARCH
# For LSTM ------------------------------------
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM
# Misc. ---------------------------------------
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

 # <strong> Phase 2: Stock Vars. Preprocessing
 ***

In [6]:
#Download all stock data

allstocks = ["TSLA", "GME", "AMD", "AAPL", "SPY"]
allstocksdata = yf.download(allstocks, start="2020-01-01", end="2025-03-17")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  5 of 5 completed


In [7]:
allstocksdata.head(5)

Price           Close                                                 High  \
Ticker           AAPL        AMD     GME         SPY       TSLA       AAPL   
Date                                                                         
2020-01-02  72.716057  49.099998  1.5775  301.194946  28.684000  72.776583   
2020-01-03  72.009132  48.599998  1.4700  298.914185  29.534000  72.771760   
2020-01-06  72.582909  48.389999  1.4625  300.054565  30.102667  72.621646   
2020-01-07  72.241547  48.250000  1.3800  299.210815  31.270666  72.849224   
2020-01-08  73.403633  47.830002  1.4300  300.805573  32.809334  73.706264   

Price                                                 ...       Open  \
Ticker            AMD     GME         SPY       TSLA  ...       AAPL   
Date                                                  ...              
2020-01-02  49.250000  1.6175  301.213507  28.713333  ...  71.721004   
2020-01-03  49.389999  1.5625  300.054558  30.266666  ...  71.941343   
2020-01-06  48.860001  1.4775  300.138003  30.104000  ...  71.127866   
2020-01-07  49.389999  1.4575  299.961784  31.441999  ...  72.592594   
2020-01-08  48.299999  1.4625  302.038636  33.232666  ...  71.943744   

Price                                                    Volume            \
Ticker            AMD     GME         SPY       TSLA       AAPL       AMD   
Date                                                                        
2020-01-02  46.860001  1.5350  299.961883  28.299999  135480400  80331100   
2020-01-03  48.029999  1.5525  297.755279  29.366667  146322800  73127400   
2020-01-06  48.020000  1.4500  297.134101  29.364668  118387200  47934900   
2020-01-07  49.349998  1.4425  299.479661  30.760000  108872000  58061400   
2020-01-08  47.849998  1.3725  299.405605  31.580000  132079200  53767000   

Price                                      
Ticker           GME       SPY       TSLA  
Date                                       
2020-01-02  17814400  59151200  142981500  
2020-01-03  14175600  77709700  266677500  
2020-01-06  13579200  55653900  151995000  
2020-01-07  20912000  40496400  268231500  
2020-01-08  22517600  68296000  467164500  

[5 rows x 25 columns]

In [8]:
''' 
This script selects numerical columns from the dataset `allstocksdata`,
standardizes their values using `StandardScaler`, and updates the dataset
with the transformed numerical data. Standardization ensures the features
have a mean of 0 and a standard deviation of 1 for improved model performance.
'''
allstocksdata.ffill(inplace=True)
numerical_cols = allstocksdata.select_dtypes(include=['float64', 'int64']).columns.tolist()
scaler = StandardScaler()
allstocksdata[numerical_cols] = scaler.fit_transform(allstocksdata[numerical_cols])

#---------------------------------------------------

''' 
Seperating th data into individual dataframes for each stock after collectiv preprocessing.
This is done to make it easier to work with the data for each stock.
'''

stock_dataframes = {}
for stock in allstocks:
    stock_dataframes[stock] = allstocksdata.xs(stock, level=1, axis=1)

TSLA_data = stock_dataframes["TSLA"] # NASDAQ
AMD_data = stock_dataframes["AMD"]   # NASDAQ
AAPL_data = stock_dataframes["AAPL"] # NASDAQ
GME_data = stock_dataframes["GME"]   # NYSE
SPY_data = stock_dataframes["SPY"]   # ARCX

 ## _Phase 2.1: Tesla, Incorporated (NASDAQ: TSLA)_

In [9]:
TSLA_data.head(5)

Price,Close,High,Low,Open,Volume
Date,,,,,
2020-01-02,-2.190752,-2.195758,-2.187560,-2.190051,0.232423
2020-01-03,-2.180945,-2.178259,-2.175587,-2.177783,1.759327
2020-01-06,-2.174384,-2.180092,-2.173163,-2.177806,0.343685
2020-01-07,-2.160909,-2.165018,-2.162646,-2.161759,1.778510
2020-01-08,-2.143157,-2.144846,-2.150940,-2.152328,4.234141


In [10]:
TSLA_data.info() #shows that date is already the index

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1307 entries, 2020-01-02 to 2025-03-14
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   1307 non-null   float64
 1   High    1307 non-null   float64
 2   Low     1307 non-null   float64
 3   Open    1307 non-null   float64
 4   Volume  1307 non-null   float64
dtypes: float64(5)
memory usage: 61.3 KB


 ## _Phase 2.2: GameStop Corporation (NYSE: GME)_

In [11]:
GME_data.head(5)

Price,Close,High,Low,Open,Volume
Date,,,,,
2020-01-02,-1.407481,-1.370269,-1.426257,-1.396096,-0.091713
2020-01-03,-1.414576,-1.373623,-1.430276,-1.394957,-0.154670
2020-01-06,-1.415071,-1.378805,-1.434471,-1.401631,-0.164989
2020-01-07,-1.420516,-1.380025,-1.437267,-1.402119,-0.038119
2020-01-08,-1.417216,-1.379720,-1.437791,-1.406677,-0.010339


 ## _Phase 2.3: Advanced Micro Devices, Incorporated (NASDAQ: AMD)_

In [12]:
AMD_data.head(5)

Price,Close,High,Low,Open,Volume
Date,,,,,
2020-01-02,-1.550585,-1.573854,-1.590573,-1.610042,0.605469
2020-01-03,-1.564716,-1.569975,-1.564377,-1.577098,0.355711
2020-01-06,-1.570651,-1.584663,-1.555165,-1.577379,-0.517736
2020-01-07,-1.574607,-1.569975,-1.549984,-1.539929,-0.166641
2020-01-08,-1.586477,-1.600183,-1.575892,-1.582166,-0.315532


 ## _Phase 2.4: Apple, Incorporated (NASDAQ: AAPL)_

In [13]:
AAPL_data.head(5)

Price,Close,High,Low,Open,Volume
Date,,,,,
2020-01-02,-1.869401,-1.894480,-1.870234,-1.890069,0.876884
2020-01-03,-1.885411,-1.894589,-1.863003,-1.885076,1.082290
2020-01-06,-1.872417,-1.897972,-1.883700,-1.903512,0.553059
2020-01-07,-1.880147,-1.892843,-1.857595,-1.870316,0.372797
2020-01-08,-1.853830,-1.873530,-1.859361,-1.885021,0.812449


 ## _Phase 2.5: SPDR S&P 500 ETF Trust (ARCX: SPY)_

In [14]:
SPY_data.head(5)

Price,Close,High,Low,Open,Volume
Date,,,,,
2020-01-02,-1.336456,-1.366067,-1.328759,-1.348854,-0.520699
2020-01-03,-1.362624,-1.379383,-1.343961,-1.374149,-0.071035
2020-01-06,-1.349540,-1.378424,-1.351828,-1.381270,-0.605437
2020-01-07,-1.359221,-1.380449,-1.331843,-1.354382,-0.972696
2020-01-08,-1.340923,-1.356587,-1.327271,-1.355231,-0.299125


 # <strong> Phase 3: Stock Vars. Prediction Modeling
 ***

In [ ]:
from Models.RY_MLEvaluation import (
    LinearReg_eval,
    AuReg_eval,
    MovAve_eval,
    ARIMA_eval,
    SARIMA_eval,
    GARCH_eval,
    LSTM_eval
) #change eval types for overall and automatic finding best prediction model

 ## _Phase 3.1: Tesla, Incorporated (NASDAQ: TSLA)_

In [75]:
''' 
Have to move it here due to constant .py file changes :p
'''

from Models.RY_MachineLearningModels import (
    LinearReg_model,
    AuReg_model,
    MovAve_model,
    ARIMA_model,
    SARIMA_model,
    GARCH_model,
    LSTM_model
)

LinearReg_model = LinearReg_model()
AuReg_model = AuReg_model()
MovAve_model = MovAve_model()
ARIMA_model = ARIMA_model()
SARIMA_model = SARIMA_model()
GARCH_model = GARCH_model()

#---------------------------------------------------

train_data = TSLA_data['Close'][:int(len(TSLA_data)*0.8)]
test_data = TSLA_data['Close'][int(len(TSLA_data)*0.8):]
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

X_train = train_data.values[:-1].reshape(-1,1)
y_train = train_data.values[1:] #targhet
X_test = test_data.values[:-1].reshape(-1,1)
y_test = test_data.values[1:]

#---------------------------------------------------

LinearReg_model.fit(X_train, y_train)
TSLA_LR_pred = LinearReg_model.predict(X_test[:10])

#---------------------------------------------------

close_series = train_data['Close']
AuReg_model.fit(train_data['Close'], lags=3)
TSLA_AR_pred = AuReg_model.predict(steps=10)

    # cant fix the valuewarnin rn, will fix later. suppress now.

import warnings
warnings.filterwarnings("ignore")

#---------------------------------------------------

MovAve_model.fit(train_data['Close'], window_size=5)
TSLA_MovAve_pred = MovAve_model.predict()
TSLA_MovAve_pred = []
window_size = 5
current_window = train_data['Close'].iloc[-window_size:].tolist()

for _ in range(10):
    next_pred = np.mean(current_window)
    TSLA_MovAve_pred.append(next_pred)
    current_window.pop(0)
    current_window.append(next_pred)

TSLA_MovAve_pred = np.array(TSLA_MovAve_pred)

#---------------------------------------------------

ARIMA_model.fit(train_data['Close'], order=(1, 1, 1))
TSLA_ARIMA_pred = ARIMA_model.predict(steps=10)

#---------------------------------------------------

SARIMA_model.fit(train_data['Close'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
TSLA_SARIMA_pred = SARIMA_model.predict(steps=10)

#---------------------------------------------------

GARCH_model.fit(train_data['Close'])
TSLA_GARCH_pred = GARCH_model.predict(steps=10)
TSLA_GARCH_pred = TSLA_GARCH_pred.mean.values[-1]

# LSTM Model ---------------------------------------

train_data_values = train_data['Close'].values #just recalling cause i didnt feel like scrolling up for edits
X_train = train_data_values[:-1].reshape(-1, 1)
y_train = train_data_values[1:]
test_data_values = test_data['Close'].values
X_test = test_data_values[:-1].reshape(-1, 1)
y_test = test_data_values[1:] 

X_train_LSTM = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_LSTM = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

input_shape = (X_train.shape[1], 1)
LSTM_model = LSTM_model(input_shape)

LSTM_model.fit(X_train_LSTM, y_train, epochs=50)
TSLA_LSTM_pred = LSTM_model.predict(X_test_LSTM)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


In [ ]:
TSLA_LR_pred = pd.DataFrame(TSLA_LR_pred, columns=['Predicted_Close'])
TSLA_AR_pred = pd.DataFrame(TSLA_AR_pred, columns=['Predicted_Close'])
TSLA_MovAve_pred = pd.DataFrame(TSLA_MovAve_pred, columns=['Predicted_Close'])
TSLA_ARIMA_pred = pd.DataFrame(TSLA_ARIMA_pred, columns=['Predicted_Close'])
TSLA_SARIMA_pred = pd.DataFrame(TSLA_SARIMA_pred, columns=['Predicted_Close'])
TSLA_GARCH_pred = pd.DataFrame(TSLA_GARCH_pred, columns=['Predicted_Close'])
TSLA_LSTM_pred = pd.DataFrame(TSLA_LSTM_pred, columns=['Predicted_Close'])



#Combine all predictions into a single dataframe; columns aree the model names, rows are the predicted values + actual values

In [ ]:
#final evaluation/find best model for TSLA automatically.

,Predicted_Close
0,-0.188259
1,-0.190092
2,-0.181387
3,-0.347464
4,-0.432220
...,...
256,0.506171
257,0.042072
258,0.138626
259,0.339178


 ## _Phase 3.2: GameStop Corporation (NYSE: GME)_

In [ ]:
''' 
Have to move it here due to constant .py file changes :p
'''

from Models.RY_MachineLearningModels import (
    LinearReg_model,
    AuReg_model,
    MovAve_model,
    ARIMA_model,
    SARIMA_model,
    GARCH_model,
    LSTM_model
)

LinearReg_model = LinearReg_model()
AuReg_model = AuReg_model()
MovAve_model = MovAve_model()
ARIMA_model = ARIMA_model()
SARIMA_model = SARIMA_model()
GARCH_model = GARCH_model()

#---------------------------------------------------

train_data = GME_data['Close'][:int(len(GME_data)*0.8)] #**
test_data = GME_data['Close'][int(len(GME_data)*0.8):] #**
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

X_train = train_data.values[:-1].reshape(-1,1)
y_train = train_data.values[1:] #targhet
X_test = test_data.values[:-1].reshape(-1,1)
y_test = test_data.values[1:]

#---------------------------------------------------

LinearReg_model.fit(X_train, y_train)
GME_LR_pred = LinearReg_model.predict(X_test[:10]) #*

#---------------------------------------------------

close_series = train_data['Close']
AuReg_model.fit(train_data['Close'], lags=3)
GME_AR_pred = AuReg_model.predict(steps=10) #*

    # cant fix the valuewarnin rn, will fix later. suppress now.

import warnings
warnings.filterwarnings("ignore")

#---------------------------------------------------

MovAve_model.fit(train_data['Close'], window_size=5)
GME_MovAve_pred = MovAve_model.predict() #*
GME_MovAve_pred = [] #*
window_size = 5
current_window = train_data['Close'].iloc[-window_size:].tolist()

for _ in range(10):
    next_pred = np.mean(current_window)
    GME_MovAve_pred.append(next_pred) #*
    current_window.pop(0)
    current_window.append(next_pred)

GME_MovAve_pred = np.array(GME_MovAve_pred) #**

#---------------------------------------------------

ARIMA_model.fit(train_data['Close'], order=(1, 1, 1))
GME_ARIMA_pred = ARIMA_model.predict(steps=10) #*

#---------------------------------------------------

SARIMA_model.fit(train_data['Close'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
GME_SARIMA_pred = SARIMA_model.predict(steps=10) #*

#---------------------------------------------------

GARCH_model.fit(train_data['Close'])
GME_GARCH_pred = GARCH_model.predict(steps=10) #*
GME_GARCH_pred = GME_GARCH_pred.mean.values[-1] #**

# LSTM Model ---------------------------------------

train_data_values = train_data['Close'].values
X_train = train_data_values[:-1].reshape(-1, 1)
y_train = train_data_values[1:]
test_data_values = test_data['Close'].values
X_test = test_data_values[:-1].reshape(-1, 1)
y_test = test_data_values[1:] 

X_train_LSTM = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_LSTM = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

input_shape = (X_train.shape[1], 1)
LSTM_model = LSTM_model(input_shape)

LSTM_model.fit(X_train_LSTM, y_train, epochs=50)
GME_LSTM_pred = LSTM_model.predict(X_test_LSTM) #*

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


 ## _Phase 3.3: Advanced Micro Devices, Incorporated (NASDAQ: AMD)_

In [71]:
''' 
Have to move it here due to constant .py file changes :p
'''

from Models.RY_MachineLearningModels import (
    LinearReg_model,
    AuReg_model,
    MovAve_model,
    ARIMA_model,
    SARIMA_model,
    GARCH_model,
    LSTM_model
)

LinearReg_model = LinearReg_model()
AuReg_model = AuReg_model()
MovAve_model = MovAve_model()
ARIMA_model = ARIMA_model()
SARIMA_model = SARIMA_model()
GARCH_model = GARCH_model()

#---------------------------------------------------

train_data = AMD_data['Close'][:int(len(AMD_data)*0.8)] #**
test_data = AMD_data['Close'][int(len(AMD_data)*0.8):] #**
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

X_train = train_data.values[:-1].reshape(-1,1)
y_train = train_data.values[1:] #targhet
X_test = test_data.values[:-1].reshape(-1,1)
y_test = test_data.values[1:]

#---------------------------------------------------

LinearReg_model.fit(X_train, y_train)
AMD_LR_pred = LinearReg_model.predict(X_test[:10]) #*

#---------------------------------------------------

close_series = train_data['Close']
AuReg_model.fit(train_data['Close'], lags=3)
AMD_AR_pred = AuReg_model.predict(steps=10) #*

    # cant fix the valuewarnin rn, will fix later. suppress now.

import warnings
warnings.filterwarnings("ignore")

#---------------------------------------------------

MovAve_model.fit(train_data['Close'], window_size=5)
AMD_MovAve_pred = MovAve_model.predict() #*
AMD_MovAve_pred = [] #*
window_size = 5
current_window = train_data['Close'].iloc[-window_size:].tolist()

for _ in range(10):
    next_pred = np.mean(current_window)
    AMD_MovAve_pred.append(next_pred) #*
    current_window.pop(0)
    current_window.append(next_pred)

AMD_MovAve_pred = np.array(AMD_MovAve_pred) #**

#---------------------------------------------------

ARIMA_model.fit(train_data['Close'], order=(1, 1, 1))
AMD_ARIMA_pred = ARIMA_model.predict(steps=10) #*

#---------------------------------------------------

SARIMA_model.fit(train_data['Close'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
AMD_SARIMA_pred = SARIMA_model.predict(steps=10) #*

#---------------------------------------------------

GARCH_model.fit(train_data['Close'])
AMD_GARCH_pred = GARCH_model.predict(steps=10) #*
AMD_GARCH_pred = AMD_GARCH_pred.mean.values[-1] #**

# LSTM Model ---------------------------------------

train_data_values = train_data['Close'].values
X_train = train_data_values[:-1].reshape(-1, 1)
y_train = train_data_values[1:]
test_data_values = test_data['Close'].values
X_test = test_data_values[:-1].reshape(-1, 1)
y_test = test_data_values[1:] 

X_train_LSTM = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_LSTM = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

input_shape = (X_train.shape[1], 1)
LSTM_model = LSTM_model(input_shape)

LSTM_model.fit(X_train_LSTM, y_train, epochs=50)
AMD_LSTM_pred = LSTM_model.predict(X_test_LSTM) #*

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


 ## _Phase 3.4: Apple, Incorporated (NASDAQ: AAPL)_

In [74]:
''' 
Have to move it here due to constant .py file changes :p
'''

from Models.RY_MachineLearningModels import (
    LinearReg_model,
    AuReg_model,
    MovAve_model,
    ARIMA_model,
    SARIMA_model,
    GARCH_model,
    LSTM_model
)

LinearReg_model = LinearReg_model()
AuReg_model = AuReg_model()
MovAve_model = MovAve_model()
ARIMA_model = ARIMA_model()
SARIMA_model = SARIMA_model()
GARCH_model = GARCH_model()

#---------------------------------------------------

train_data = AAPL_data['Close'][:int(len(AAPL_data)*0.8)] #**
test_data = AAPL_data['Close'][int(len(AAPL_data)*0.8):] #**
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

X_train = train_data.values[:-1].reshape(-1,1)
y_train = train_data.values[1:] #targhet
X_test = test_data.values[:-1].reshape(-1,1)
y_test = test_data.values[1:]

#---------------------------------------------------

LinearReg_model.fit(X_train, y_train)
AAPL_LR_pred = LinearReg_model.predict(X_test[:10]) #*

#---------------------------------------------------

close_series = train_data['Close']
AuReg_model.fit(train_data['Close'], lags=3)
AAPL_AR_pred = AuReg_model.predict(steps=10) #*

    # cant fix the valuewarnin rn, will fix later. suppress now.

import warnings
warnings.filterwarnings("ignore")

#---------------------------------------------------

MovAve_model.fit(train_data['Close'], window_size=5)
AAPL_MovAve_pred = MovAve_model.predict() #*
AAPL_MovAve_pred = [] #*
window_size = 5
current_window = train_data['Close'].iloc[-window_size:].tolist()

for _ in range(10):
    next_pred = np.mean(current_window)
    AAPL_MovAve_pred.append(next_pred) #*
    current_window.pop(0)
    current_window.append(next_pred)

AAPL_MovAve_pred = np.array(AAPL_MovAve_pred) #*

#---------------------------------------------------

ARIMA_model.fit(train_data['Close'], order=(1, 1, 1))
AAPL_ARIMA_pred = ARIMA_model.predict(steps=10) #*

#---------------------------------------------------

SARIMA_model.fit(train_data['Close'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
AAPL_SARIMA_pred = SARIMA_model.predict(steps=10) #*

#---------------------------------------------------

GARCH_model.fit(train_data['Close'])
AAPL_GARCH_pred = GARCH_model.predict(steps=10) #*
AAPL_GARCH_pred = AAPL_GARCH_pred.mean.values[-1] #**

# LSTM Model ---------------------------------------

train_data_values = train_data['Close'].values
X_train = train_data_values[:-1].reshape(-1, 1)
y_train = train_data_values[1:]
test_data_values = test_data['Close'].values
X_test = test_data_values[:-1].reshape(-1, 1)
y_test = test_data_values[1:] 

X_train_LSTM = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_LSTM = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

input_shape = (X_train.shape[1], 1)
LSTM_model = LSTM_model(input_shape)

LSTM_model.fit(X_train_LSTM, y_train, epochs=50)
AAPL_LSTM_pred = LSTM_model.predict(X_test_LSTM) #*

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


 ## _Phase 3.5: SPDR S&P 500 ETF Trust (ARCX: SPY)_

In [ ]:
''' 
Have to move it here due to constant .py file changes :p
'''

from Models.RY_MachineLearningModels import (
    LinearReg_model,
    AuReg_model,
    MovAve_model,
    ARIMA_model,
    SARIMA_model,
    GARCH_model,
    LSTM_model
)

LinearReg_model = LinearReg_model()
AuReg_model = AuReg_model()
MovAve_model = MovAve_model()
ARIMA_model = ARIMA_model()
SARIMA_model = SARIMA_model()
GARCH_model = GARCH_model()

#---------------------------------------------------

train_data = SPY_data['Close'][:int(len(SPY_data)*0.8)] #**
test_data = SPY_data['Close'][int(len(SPY_data)*0.8):] #**
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

X_train = train_data.values[:-1].reshape(-1,1)
y_train = train_data.values[1:] #targhet
X_test = test_data.values[:-1].reshape(-1,1)
y_test = test_data.values[1:]

#---------------------------------------------------

LinearReg_model.fit(X_train, y_train)
SPY_LR_pred = LinearReg_model.predict(X_test[:10]) #*

#---------------------------------------------------

close_series = train_data['Close']
AuReg_model.fit(train_data['Close'], lags=3)
SPY_AR_pred = AuReg_model.predict(steps=10) #*

    # cant fix the valuewarnin rn, will fix later. suppress now.

import warnings
warnings.filterwarnings("ignore")

#---------------------------------------------------

MovAve_model.fit(train_data['Close'], window_size=5)
SPY_MovAve_pred = MovAve_model.predict() #*
SPY_MovAve_pred = [] #*
window_size = 5
current_window = train_data['Close'].iloc[-window_size:].tolist()

for _ in range(10):
    next_pred = np.mean(current_window)
    SPY_MovAve_pred.append(next_pred) #*
    current_window.pop(0)
    current_window.append(next_pred)

SPY_MovAve_pred = np.array(SPY_MovAve_pred) #**

#---------------------------------------------------

ARIMA_model.fit(train_data['Close'], order=(1, 1, 1))
SPY_ARIMA_pred = ARIMA_model.predict(steps=10) #*

#---------------------------------------------------

SARIMA_model.fit(train_data['Close'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
SPY_SARIMA_pred = SARIMA_model.predict(steps=10) #*

#---------------------------------------------------

GARCH_model.fit(train_data['Close'])
SPY_GARCH_pred = GARCH_model.predict(steps=10) #*
SPY_GARCH_pred = SPY_GARCH_pred.mean.values[-1] #**

# LSTM Model ---------------------------------------

train_data_values = train_data['Close'].values
X_train = train_data_values[:-1].reshape(-1, 1)
y_train = train_data_values[1:]
test_data_values = test_data['Close'].values
X_test = test_data_values[:-1].reshape(-1, 1)
y_test = test_data_values[1:] 

X_train_LSTM = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_LSTM = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

input_shape = (X_train.shape[1], 1)
LSTM_model = LSTM_model(input_shape)

LSTM_model.fit(X_train_LSTM, y_train, epochs=50)
SPY_LSTM_pred = LSTM_model.predict(X_test_LSTM) #*

In [ ]:
# LSTM Model ------------------------------------

train_data_values = train_data['Close'].values
X_train = train_data_values[:-1].reshape(-1, 1)
y_train = train_data_values[1:]
test_data_values = test_data['Close'].values
X_test = test_data_values[:-1].reshape(-1, 1)
y_test = test_data_values[1:] 

X_train_LSTM = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_LSTM = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

input_shape = (X_train.shape[1], 1)
LSTM_model = LSTM_model(input_shape)

LSTM_model.fit(X_train_LSTM, y_train, epochs=50)
SPY_LSTM_pred = LSTM_model.predict(X_test_LSTM)

 # <strong> Phase 4: Findings and Summary
 ***

 ## _Phase 4.1: Tesla, Incorporated (NASDAQ: TSLA)_

 ## _Phase 4.2: GameStop Corporation (NYSE: GME)_

 ## _Phase 4.3: Advanced Micro Devices, Incorporated (NASDAQ: AMD)_

 ## _Phase 4.4: Apple, Incorporated (NASDAQ: AAPL)_

 ## _Phase 4.5: SPDR S&P 500 ETF Trust (ARCX: SPY)_